In [322]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [323]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [324]:
customers = pd.read_csv('customers.csv')
campaigns = pd.read_csv('marketing_campaigns.csv')
spend = pd.read_csv('marketing_spend.csv')
b_customer = pd.read_csv('b_customer.csv')
b_customer_tp = pd.read_csv('b_customer_touchpoints.csv')
b_marketing_campaigns = pd.read_csv('b_marketing_campaign.csv')
b_transactions = pd.read_csv('b_trx.csv')
transactions_v2 = pd.read_csv('transactions_v2.csv')
customer_tp_v2 = pd.read_csv('customer_touchpoints_v2.csv')
omni = pd.read_csv('omnichannel-dataset.csv')

In [325]:
print('Data Shapes:\n')
print(f'customers: {customers.shape}')
print(f'campaigns: {campaigns.shape}')
print(f'spend: {spend.shape}')
print(f'b_customer: {b_customer.shape}')
print(f'b_customer_tp: {b_customer_tp.shape}')
print(f'b_marketing_campaigns: {b_marketing_campaigns.shape}')
print(f'b_transactions: {b_transactions.shape}')
print(f'transactions_v2: {transactions_v2.shape}')
print(f'customer_tp_v2: {customer_tp_v2.shape}')
print(f'omni: {omni.shape}')

Data Shapes:

customers: (31, 13)
campaigns: (30, 10)
spend: (30, 9)
b_customer: (100, 13)
b_customer_tp: (3894, 17)
b_marketing_campaigns: (30, 10)
b_transactions: (3072, 16)
transactions_v2: (45, 16)
customer_tp_v2: (27, 17)
omni: (100, 30)


In [326]:
# Create dictionary of missing value counts for each dataset
missings = {
    'customers': customers.isnull().sum(),
    'campaigns': campaigns.isnull().sum(),
    'spend': spend.isnull().sum(),
    'b_customer': b_customer.isnull().sum(),
    'b_customer_tp': b_customer_tp.isnull().sum(),
    'b_marketing_campaigns': b_marketing_campaigns.isnull().sum(),
    'b_transactions': b_transactions.isnull().sum(),
    'transactions_v2': transactions_v2.isnull().sum(),
    'customer_tp_v2': customer_tp_v2.isnull().sum(),
    'omni': omni.isnull().sum(),
}

print('Missing Values:')
for name, series in missings.items():
    # Filter to only columns with missing values
    nonzero = series[series > 0]
    if nonzero.any():
        print(f'\n{name}:')
        print(nonzero.to_string())

Missing Values:

b_customer_tp:
CAMPAIGN_ID        3576
REFERRER_SOURCE    3894
LANDING_PAGE       2465
TRANSACTION_ID      822
DEVICE_TYPE        2351
STORE_ID           1543

b_transactions:
STORE_ID            721
CARD_BANK          3072
PROMO_CODE_USED    3072
DEVICE_TYPE        2351

transactions_v2:
STORE_ID           29
CARD_BANK          41
SHIPPING_AMOUNT     3
PROMO_CODE_USED    26
DEVICE_TYPE        16

customer_tp_v2:
CAMPAIGN_ID        12
REFERRER_SOURCE     5
LANDING_PAGE        8
PAGES_VIEWED        8
PRODUCTS_VIEWED     7
TRANSACTION_ID      8
DEVICE_TYPE         5
STORE_ID           22

omni:
cart_abandonment_stage    82


In [327]:
# Drop CARD_BANK columns
b_transactions = b_transactions.drop(columns = ['CARD_BANK'], axis=1)
transactions_v2 = transactions_v2.drop(columns = ['CARD_BANK'], axis=1)

# Fill missing values in b_transactions and transactions_v2
b_transactions['SHIPPING_AMOUNT'] = b_transactions['SHIPPING_AMOUNT'].fillna(b_transactions['SHIPPING_AMOUNT'].mean())
b_transactions['PROMO_CODE_USED'] = b_transactions['PROMO_CODE_USED'].fillna('N/A')
b_transactions['STORE_ID'] = b_transactions['STORE_ID'].fillna('N/A')
b_transactions['DEVICE_TYPE'] = b_transactions['DEVICE_TYPE'].fillna('InStore')

transactions_v2['SHIPPING_AMOUNT'] = transactions_v2['SHIPPING_AMOUNT'].fillna(transactions_v2['SHIPPING_AMOUNT'].mean())
transactions_v2['PROMO_CODE_USED'] = transactions_v2['PROMO_CODE_USED'].fillna('N/A')
transactions_v2['STORE_ID'] = transactions_v2['STORE_ID'].fillna('N/A')
transactions_v2['DEVICE_TYPE'] = transactions_v2['DEVICE_TYPE'].fillna('InStore')

omni['cart_abandonment_stage'] = omni['cart_abandonment_stage'].fillna('N/A')

In [328]:
# Fill missing values in b_customer_tp and customer_tp_v2
b_customer_tp['CAMPAIGN_ID'] = b_customer_tp['CAMPAIGN_ID'].fillna("N/A")
b_customer_tp.drop('REFERRER_SOURCE', axis=1, inplace=True)
b_customer_tp.drop('LANDING_PAGE', axis=1, inplace=True)
b_customer_tp['PAGES_VIEWED'] = b_customer_tp['PAGES_VIEWED'].fillna('N/A')
b_customer_tp.drop('PRODUCTS_VIEWED', axis=1, inplace=True)
b_customer_tp['TRANSACTION_ID'] = b_customer_tp['TRANSACTION_ID'].fillna('N/A')
b_customer_tp['DEVICE_TYPE'] = b_customer_tp['DEVICE_TYPE'].fillna('InStore')
b_customer_tp['STORE_ID'] = b_customer_tp['STORE_ID'].fillna('N/A')

customer_tp_v2['CAMPAIGN_ID'] = customer_tp_v2['CAMPAIGN_ID'].fillna("N/A")
customer_tp_v2.drop('REFERRER_SOURCE', axis=1, inplace=True)
customer_tp_v2.drop('LANDING_PAGE', axis=1, inplace=True)
customer_tp_v2['PAGES_VIEWED'] = customer_tp_v2['PAGES_VIEWED'].fillna('N/A')
customer_tp_v2.drop('PRODUCTS_VIEWED', axis=1, inplace=True)
customer_tp_v2['TRANSACTION_ID'] = customer_tp_v2['TRANSACTION_ID'].fillna('N/A')
customer_tp_v2['DEVICE_TYPE'] = customer_tp_v2['DEVICE_TYPE'].fillna('InStore')
customer_tp_v2['STORE_ID'] = customer_tp_v2['STORE_ID'].fillna('N/A')

In [329]:
# Recheck missing values
missings = {
    'customers': customers.isnull().sum(),
    'campaigns': campaigns.isnull().sum(),
    'spend': spend.isnull().sum(),
    'b_customer': b_customer.isnull().sum(),
    'b_customer_tp': b_customer_tp.isnull().sum(),
    'b_marketing_campaigns': b_marketing_campaigns.isnull().sum(),
    'b_transactions': b_transactions.isnull().sum(),
    'transactions_v2': transactions_v2.isnull().sum(),
    'customer_tp_v2': customer_tp_v2.isnull().sum(),
    'omni': omni.isnull().sum(),
}
print('Missing Values:')
for name, series in missings.items():
    # Filter to only columns with missing values
    nonzero = series[series > 0]
    if nonzero.any():
        print(f'\n{name}:')
        print(nonzero.to_string())

Missing Values:


In [331]:
# Check for duplicated rows in each DataFrame and print a short message if none found
datasets = {
    'customers': customers,
    'campaigns': campaigns,
    'spend': spend,
    'b_customer': b_customer,
    'b_customer_tp': b_customer_tp,
    'b_marketing_campaigns': b_marketing_campaigns,
    'b_transactions': b_transactions,
    'transactions_v2': transactions_v2,
    'customer_tp_v2': customer_tp_v2,
    'omni': omni,
}
for name, df in datasets.items():
    dup = df.loc[df.duplicated()]
    print(f'{name}:')
    if dup.empty:
        print('  No duplicate rows')
    else:
        print(dup)
    print()

customers:
    customer_id first_name last_name             email     city state  age  \
30         1030       Eric     Young  eric.y@email.com  Houston    TX   46   

   gender customer_segment acquisition_date acquisition_channel  \
30      M          Dormant          12/1/20               Email   

    lifetime_value churn_risk  
30          267.90       High  

campaigns:
  No duplicate rows

spend:
  No duplicate rows

b_customer:
  No duplicate rows

b_customer_tp:
  No duplicate rows

b_marketing_campaigns:
  No duplicate rows

b_transactions:
  No duplicate rows

transactions_v2:
  No duplicate rows

customer_tp_v2:
  No duplicate rows

omni:
  No duplicate rows



In [332]:
customers.drop_duplicates(keep = 'first', inplace = True) 

In [341]:
# Check for duplicated rows in each DataFrame and print a short message if none found
datasets = {
    'customers': customers,
    'campaigns': campaigns,
    'spend': spend,
    'b_customer': b_customer,
    'b_customer_tp': b_customer_tp,
    'b_marketing_campaigns': b_marketing_campaigns,
    'b_transactions': b_transactions,
    'transactions_v2': transactions_v2,
    'customer_tp_v2': customer_tp_v2,
    'omni': omni,
}
for name, df in datasets.items():
    dup = df.loc[df.duplicated()]
    print(f'{name}:')
    if dup.empty:
        print('  No duplicate rows')
    else:
        print(dup)
    print()

customers:
  No duplicate rows

campaigns:
  No duplicate rows

spend:
  No duplicate rows

b_customer:
  No duplicate rows

b_customer_tp:
  No duplicate rows

b_marketing_campaigns:
  No duplicate rows

b_transactions:
  No duplicate rows

transactions_v2:
  No duplicate rows

customer_tp_v2:
  No duplicate rows

omni:
  No duplicate rows



In [342]:
print('Data Shapes:\n')
print(f'customers: {customers.shape}')
print(f'campaigns: {campaigns.shape}')
print(f'spend: {spend.shape}')
print(f'b_customer: {b_customer.shape}')
print(f'b_customer_tp: {b_customer_tp.shape}')
print(f'b_marketing_campaigns: {b_marketing_campaigns.shape}')
print(f'b_transactions: {b_transactions.shape}')
print(f'transactions_v2: {transactions_v2.shape}')
print(f'customer_tp_v2: {customer_tp_v2.shape}')
print(f'omni: {omni.shape}')

Data Shapes:

customers: (30, 13)
campaigns: (30, 10)
spend: (30, 9)
b_customer: (100, 13)
b_customer_tp: (3894, 14)
b_marketing_campaigns: (30, 10)
b_transactions: (3072, 15)
transactions_v2: (45, 15)
customer_tp_v2: (27, 14)
omni: (100, 30)


In [343]:
# Standardize acquisition date format for customers and b_customer to YYYY-MM-DD
# This cell assumes pandas is already imported and `customers` and `b_customer` exist.

def _standardize_acq_date(df, preferred_format='%Y-%m-%d'):
    # possible column name variants
    candidates = ['acquisition_date', 'ACQUISITION_DATE', 'Acquisition_Date']
    col = next((c for c in candidates if c in df.columns), None)
    if col is None:
        print(f'No acquisition date column found in DataFrame with columns: {list(df.columns)}')
        return None

    s = df[col].astype(str)

    # try common specific formats first, then fall back to inference
    parsed = pd.to_datetime(s, format='%m/%d/%y', errors='coerce')   # e.g. 3/15/22
    parsed = parsed.combine_first(pd.to_datetime(s, format='%m/%d/%Y', errors='coerce'))
    parsed = parsed.combine_first(pd.to_datetime(s, format='%Y-%m-%d', errors='coerce'))  # e.g. 2023-11-24
    parsed = parsed.combine_first(pd.to_datetime(s, infer_datetime_format=True, errors='coerce'))

    # report if any rows could not be parsed
    if parsed.isna().any():
        n_unparsed = parsed.isna().sum()
        print(f'Warning: {n_unparsed} values in "{col}" could not be parsed to datetime.')

    # set column to consistent string format (YYYY-MM-DD) and keep as datetime as well
    df[col] = parsed.dt.strftime(preferred_format)
    # If you prefer to keep as datetime dtype instead of strings, uncomment:
    # df[col] = parsed

    return col

for name, df in (('customers', customers), ('b_customer', b_customer)):
    col = _standardize_acq_date(df)
    if col:
        print(f'{name}.{col} -> sample:\n', df[col].head().to_string(index=False))

customers.ACQUISITION_DATE -> sample:
 2022-03-15
2021-01-20
2021-06-10
2023-02-28
2024-01-05
b_customer.ACQUISITION_DATE -> sample:
 2025-08-24
2023-11-24
2025-07-10
2024-12-14
2024-01-24


In [344]:
# Standardize order/transaction date to YYYY-MM-DD (drop time portion)
def _standardize_order_date(df, preferred_format='%Y-%m-%d'):
    candidates = ['order_date', 'ORDER_DATE', 'Order_Date']
    col = next((c for c in candidates if c in df.columns), None)
    if col is None:
        print(f'No order date column found in DataFrame with columns: {list(df.columns)}')
        return None

    s = df[col].astype(str)

    # try a list of common formats, then fall back to inference
    formats = [
        '%m/%d/%y %H:%M',
        '%m/%d/%Y %H:%M',
        '%m/%d/%y',
        '%m/%d/%Y',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d'
    ]

    parsed = pd.to_datetime(s, errors='coerce')
    for fmt in formats:
        parsed = parsed.combine_first(pd.to_datetime(s, format=fmt, errors='coerce'))

    parsed = parsed.combine_first(pd.to_datetime(s, infer_datetime_format=True, errors='coerce'))

    if parsed.isna().any():
        n_unparsed = parsed.isna().sum()
        print(f'Warning: {n_unparsed} values in "{col}" could not be parsed to datetime.')

    # set to YYYY-MM-DD (string). If you prefer datetime dtype, assign parsed instead.
    df[col] = parsed.dt.strftime(preferred_format)

    return col

for name, df in (('b_transactions', b_transactions), ('transactions_v2', transactions_v2)):
    col = _standardize_order_date(df)
    if col:
        print(f'{name}.{col} -> sample:\n', df[col].head().to_string(index=False))

b_transactions.ORDER_DATE -> sample:
 2024-07-14
2024-07-15
2024-07-17
2024-07-29
2024-08-21
transactions_v2.ORDER_DATE -> sample:
 2024-01-05
2024-01-12
2024-01-08
2024-01-15
2024-01-07


In [345]:
def _standardize_touchpoint_timestamp(df, preferred_format='%Y-%m-%d'):
    candidates = ['touchpoint_timestamp', 'TOUCHPOINT_TIMESTAMP', 'Touchpoint_Timestamp']
    col = next((c for c in candidates if c in df.columns), None)
    if col is None:
        print(f'No touchpoint timestamp column found in DataFrame with columns: {list(df.columns)}')
        return None

    s = df[col].astype(str)

    # try common formats then fall back to inference
    formats = [
        '%m/%d/%y %H:%M',
        '%m/%d/%Y %H:%M',
        '%m/%d/%y %I:%M %p',
        '%m/%d/%y %H:%M:%S',
        '%m/%d/%Y %H:%M:%S',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d'
    ]

    parsed = pd.to_datetime(s, errors='coerce')
    for fmt in formats:
        parsed = parsed.combine_first(pd.to_datetime(s, format=fmt, errors='coerce'))

    parsed = parsed.combine_first(pd.to_datetime(s, infer_datetime_format=True, errors='coerce'))

    if parsed.isna().any():
        print(f'Warning: {parsed.isna().sum()} values in "{col}" could not be parsed to datetime.')

    # set to YYYY-MM-DD strings
    df[col] = parsed.dt.strftime(preferred_format)
    return col

for name, df in (('b_customer_tp', b_customer_tp), ('customer_tp_v2', customer_tp_v2)):
    col = _standardize_touchpoint_timestamp(df)
    if col:
        print(f'{name}.{col} -> sample:\n', df[col].head().to_string(index=False))

b_customer_tp.TOUCHPOINT_TIMESTAMP -> sample:
 2024-07-14
2024-07-15
2024-07-17
2024-07-29
2024-08-21
customer_tp_v2.TOUCHPOINT_TIMESTAMP -> sample:
 2024-01-03
2024-01-04
2024-01-07
2024-01-05
2024-01-08


In [346]:
# Capitalize column headings for every DataFrame in the `datasets` dict (in-place)
for name, df in datasets.items():
    df.columns = [str(c).upper() for c in df.columns]
    # ensure top-level variable references (if present) point to the updated DataFrame
    if name in globals():
        globals()[name] = df

print("Capitalized column names for datasets:", ", ".join(datasets.keys()))

Capitalized column names for datasets: customers, campaigns, spend, b_customer, b_customer_tp, b_marketing_campaigns, b_transactions, transactions_v2, customer_tp_v2, omni


In [347]:
def smart_merge_tables(df_main: pd.DataFrame, df_other: pd.DataFrame,
                       assign_to_main: bool = False, main_name: str | None = None) -> pd.DataFrame:
    """
    Minimal merge helper:
    - If the two DataFrames have different column SETs -> raise ValueError (lists differences).
    - If the same set but different order -> reorder df_other to match df_main.
    - Concatenate the two DataFrames, drop exact duplicate rows, reset index, and return.
    - If assign_to_main=True and main_name provided, assign result to globals()[main_name].
    """
    cols_main = list(df_main.columns)
    cols_other = list(df_other.columns)

    # Check column sets
    if set(cols_main) != set(cols_other):
        only_in_main = [c for c in cols_main if c not in cols_other]
        only_in_other = [c for c in cols_other if c not in cols_main]
        raise ValueError(f"Cannot merge — column sets differ. Only in main: {only_in_main}; Only in other: {only_in_other}")

    # Reorder other if needed
    if cols_main != cols_other:
        df_other = df_other[cols_main]

    merged = pd.concat([df_main, df_other], ignore_index=True)
    before = merged.shape[0]
    merged = merged.drop_duplicates(keep='first').reset_index(drop=True)
    dropped = before - merged.shape[0]

    if assign_to_main and main_name:
        globals()[main_name] = merged

    print(f"Merged -> shape: {merged.shape} (dropped {dropped} duplicate rows)")
    return merged

In [305]:
smart_merge_tables(customers, b_customer, assign_to_main=True, main_name='clean_customers')
smart_merge_tables(b_transactions, transactions_v2, assign_to_main=True, main_name='clean_transactions')
smart_merge_tables(b_customer_tp, customer_tp_v2, assign_to_main=True, main_name='clean_touchpoints')
smart_merge_tables(b_marketing_campaigns, campaigns, assign_to_main=True, main_name='clean_campaigns')

Merged -> shape: (130, 13) (dropped 0 duplicate rows)
Merged -> shape: (3117, 15) (dropped 0 duplicate rows)
Merged -> shape: (3921, 14) (dropped 0 duplicate rows)
Merged -> shape: (60, 10) (dropped 0 duplicate rows)


,CAMPAIGN_ID,CAMPAIGN_NAME,CAMPAIGN_TYPE,CHANNEL,START_DATE,END_DATE,BUDGET,TARGET_AUDIENCE,CAMPAIGN_GOAL,STATUS
0,129,Last Chance Email,Sales,Email,0024-07-01,0024-07-31,400,Inactive Users,Reactivation,Completed
1,125,Viral Challenge,Influencer,TikTok,0024-07-01,0024-07-31,38000,18-25 All,Brand Awareness,Completed
2,130,Push Power Hour,Product Launch,Push Notification,0024-08-01,0024-08-20,8000,App Users,Engagement,Completed
3,126,Love Your Look,Sales,Instagram,0024-08-01,0024-08-20,16000,All Segments,Revenue,Completed
4,127,Dynamic Remarketing,Performance,Google,0024-09-01,0024-09-30,28000,Site Visitors,Conversion,Completed
5,111,Summer Preview,Promotional,Instagram,0024-10-01,0024-11-20,22000,18-35 Female,Brand Awareness,Completed
6,114,Athleisure Campaign,Promotional,Facebook,0024-10-01,0024-11-20,10000,25-45 Active,Acquisition,Completed
7,128,Community Event,Event,Direct Mail,0024-10-01,0024-11-20,3500,Local Residents,Traffic,Completed
8,112,Flash Sale Friday,Sales,Google,0024-11-01,0025-01-05,8000,All Segments,Revenue,Completed
9,113,Loyalty Points Bonus,Retention,Email,0024-12-01,0025-01-24,800,High-Value Customers,Engagement,Completed


In [348]:
from pathlib import Path

out_dir = Path('clean_data')
out_dir.mkdir(parents=True, exist_ok=True)

exports = {
    'clean_touchpoints': clean_touchpoints,
    'clean_campaigns': clean_campaigns,
    'clean_customers': clean_customers,
    'clean_spend': spend,
    'clean_transactions': clean_transactions,
    'clean_omni': omni,
}

for name, df in exports.items():
    path = out_dir / f"{name}.csv"
    df.to_csv(path, index=False)
    print(f"Exported {name} -> {path}")

Exported clean_touchpoints -> clean_data\clean_touchpoints.csv
Exported clean_campaigns -> clean_data\clean_campaigns.csv
Exported clean_customers -> clean_data\clean_customers.csv
Exported clean_spend -> clean_data\clean_spend.csv
Exported clean_transactions -> clean_data\clean_transactions.csv
Exported clean_omni -> clean_data\clean_omni.csv


In [352]:
!pip install gc
from gc import freeze

!pip freeze > requirements.txt

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement gc (from versions: none)
ERROR: No matching distribution found for gc
